<a href="https://colab.research.google.com/github/Moore-Lab-UMass/ENCODE-API/blob/main/Pull_TF_Bams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Jill E. Moore
#Moore Lab - UMass Chan Medical School
#May 2022

import os
import json
import urllib
from urllib import request, parse, error
import requests

In [2]:
def Download_File(name, extension, outputDir): #downloads file to local machine
    if not os.path.exists(outputDir+"/"+name+"."+extension):
        url="https://www.encodeproject.org/files/"+name+"/@@download/"+name+"."+extension
        r = requests.get(url)
        outputFile=open(outputDir+"/"+name+"."+extension, "wb")
        outputFile.write(r.content)
        outputFile.close()

In [19]:
def Extract_Experiment_Metatdata(exp, genome): #extracts experiment metadata
  urlExp = "https://www.encodeproject.org/experiments/"+exp+"/?format=json"
  response = urllib.request.urlopen(urlExp)
  data = json.loads(response.read())

  rfa = data["award"]["rfa"] #project/phase of encode
  lab = data["lab"]["title"] #lab that submitted the experiment
  biosample = data["biosample_ontology"]["term_name"] #short name of biosample
  tf = data["target"]["label"] #target of the assay, i.e. TF

  bam = "NA"
  numReads = "NA"
  replicate = "NA"

  for entry in data["files"]: #loops through files associated with experiment
    if entry["file_type"] == "bam" and entry["status"] == "released" \
      and entry["assembly"] == genome and entry["output_type"] == "alignments":

      bam = entry["accession"] #bam file
      #Download_File(bam, "bam", "/data/output")
      replicate = ",".join([str(i) for i in entry["biological_replicates"]])

      for qc in entry["quality_metrics"]:
        if "mapped_reads" in qc:
          numReads = qc["mapped_reads"]

      for pipeline in entry["analyses"]:
        if pipeline["status"] == "released":
          print(exp + "\t" + bam +"\t" + replicate + "\t" + biosample + "\t"+ \
                tf +"\t" + rfa +"\t"+ lab + "\t" + str(numReads))


In [22]:
## Parameters
genome = "GRCh38"
species = "Homo+sapiens"

## Build query
urlMain = "https://www.encodeproject.org/search/?type=Experiment&status=released" + \
    "&perturbed=false&perturbed=true&assay_title=TF+ChIP-seq" + \
    "&replicates.library.biosample.donor.organism.scientific_name=" + species + \
    "&format=json&limit=50" #update limit=all for all datasets

response = urllib.request.urlopen(urlMain)
data = json.loads(response.read())

print("experiment_accession" + "\t" + "bam_accession" + "\t" + "replicate" + \
      "\t" + "biosample" + "\t" + "tf" + "\t" + "rfa" +"\t"+ "lab" + "\t" + \
      "number_reads")

for entry in data["@graph"]: #loops through experiments
  Extract_Experiment_Metatdata(entry["accession"], genome)

experiment_accession	bam_accession	replicate	biosample	tf	rfa	lab	number_reads
ENCSR506SSQ	ENCFF280CMY	1	K562	ZNF354C	ENCODE4	Michael Snyder, Stanford	58697182
ENCSR506SSQ	ENCFF682BHG	2	K562	ZNF354C	ENCODE4	Michael Snyder, Stanford	57255972
ENCSR423MQG	ENCFF728WAR	1	brain	CTCF	ENCODE4	Bradley Bernstein, Broad	18384265
ENCSR427WZJ	ENCFF824EZV	2	K562	ZNF143	ENCODE4	Richard Myers, HAIB	23929372
ENCSR427WZJ	ENCFF115IEU	1	K562	ZNF143	ENCODE4	Richard Myers, HAIB	28121139
ENCSR159RBE	ENCFF940GKH	2	HepG2	GLYR1	ENCODE4	Richard Myers, HAIB	37776022
ENCSR159RBE	ENCFF438DBB	1	HepG2	GLYR1	ENCODE4	Richard Myers, HAIB	25896403
ENCSR015LYB	ENCFF741MBT	2	HepG2	HIC2	ENCODE4	Richard Myers, HAIB	19921844
ENCSR015LYB	ENCFF678WPD	1	HepG2	HIC2	ENCODE4	Richard Myers, HAIB	34446310
ENCSR279OXE	ENCFF026KCW	1	K562	ZFP30	ENCODE4	Michael Snyder, Stanford	46192754
ENCSR279OXE	ENCFF573IHA	2	K562	ZFP30	ENCODE4	Michael Snyder, Stanford	86042260
ENCSR208VNN	ENCFF748UJX	1	K562	IFI16	ENCODE4	Michael Snyder, Stanford	5445